In [48]:
# Library Import
import boto3
import pandas as pd
from io import StringIO


In [49]:
# Enter your credentials for AWS connection
AWS_ACCESS_KEY = "YOUR_AWS_ACCESS_KEY"
AWS_SECRET_KEY = "YOUR_AWS_SECRET_KEY"
AWS_REGION = "us-east-1"
SCHEMA_NAME = "test1"
S3_STAGING_DIR = "s3://bucket-name/filename/"
S3_BUCKET_NAME = "bucket-name"
S3_OUTPUT_DIRECTORY = "filename"

In [50]:
#AWS Athena connection via boto3
athena_client = boto3.client(
    "athena",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)

In [51]:
# Run SELECT query
database = SCHEMA_NAME
table = "enigma_d"
query_string = "SELECT * FROM {}.{};".format(database, table)

In [52]:
# Running Query on the mentioned database.
response = athena_client.start_query_execution(
    QueryString=query_string,
    QueryExecutionContext={
        'Database': database
    },
    ResultConfiguration={
        'OutputLocation': S3_STAGING_DIR
    }
)
query_execution_id = response['QueryExecutionId']
print("Query execution ID:", query_execution_id)

Query execution ID: 2d8fc518-f338-44cb-b0c4-122531bb921c


In [53]:
# Checking for Query Status
query_status = None
while query_status == 'QUEUED' or query_status == 'RUNNING' or query_status is None:
    query_status = athena_client.get_query_execution(QueryExecutionId=query_execution_id)['QueryExecution']['Status']['State']
    if query_status == 'FAILED' or query_status == 'CANCELLED':
        raise Exception("Athena query failed or was cancelled")
    elif query_status == 'SUCCEEDED':
        print("Query execution completed successfully")
        break

Query execution completed successfully


In [54]:
# Creating connection and reading the data-file as a csv and storing in the Dataframe.
temp_file_loc: str = "file_data.csv"
s3_client = boto3.client(
    "s3",
    aws_access_key_id=AWS_ACCESS_KEY,
    aws_secret_access_key=AWS_SECRET_KEY,
    region_name=AWS_REGION
)
s3_client.download_file(
    S3_BUCKET_NAME,
    f"{S3_OUTPUT_DIRECTORY}/{query_execution_id}.csv",
    temp_file_loc
)
df = pd.read_csv('file_data.csv')

In [55]:
df.head()

,fips,admin2,province_state,country_region,last_update,latitude,longitude,confirmed,deaths,recovered,active,combined_key
0,NaN,NaN,Anhui,China,2020-01-22T17:00:00,31.826,117.226,1.0,NaN,NaN,NaN,"""Anhui"
1,NaN,NaN,Beijing,China,2020-01-22T17:00:00,40.182,116.414,14.0,NaN,NaN,NaN,"""Beijing"
2,NaN,NaN,Chongqing,China,2020-01-22T17:00:00,30.057,107.874,6.0,NaN,NaN,NaN,"""Chongqing"
3,NaN,NaN,Fujian,China,2020-01-22T17:00:00,26.079,117.987,1.0,NaN,NaN,NaN,"""Fujian"
4,NaN,NaN,Gansu,China,2020-01-22T17:00:00,36.061,103.834,NaN,NaN,NaN,NaN,"""Gansu"
